In [ ]:
%pip install nest_asyncio uvicorn fastapi python-socketio

In [1]:
import asyncio
import threading
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import socketio
import socket
import subprocess
import os
import sys

sio = socketio.AsyncServer(async_mode='asgi', cors_allowed_origins='*')

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 將 Socket.IO 服務器掛載到 FastAPI 應用上
socket_app = socketio.ASGIApp(sio, other_asgi_app=app)

async def async_emit_message(topic, msg):
    await sio.emit(topic, msg)

def emit_message(topic, msg):
    print(f'[Socket] {topic}.emit: {msg}')
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    if loop.is_running():
        asyncio.ensure_future(async_emit_message(topic, msg))
    else:
        loop.run_until_complete(async_emit_message(topic, msg))

# 身份驗證中間件
@sio.event
async def connect(sid, environ, auth=None):
    print("Client connect", sid)
    query_string = environ.get('QUERY_STRING', '')
    query_params = dict(param.split('=') for param in query_string.split('&') if '=' in param)
    token = query_params.get('token', None)

    if token == "UNITY":
        await sio.save_session(sid, {"authenticated": True})
        return True  # 允許連接

    await sio.disconnect(sid)
    return False  # 斷開連接

@sio.event
async def disconnect(sid):
    print("Client disconnected", sid)

@sio.on('asr_toggle')
async def handleASRToggle(sid, data):
    print('handleASRToggle', data)
    state = 0 if data else 1
    print('asr_toggle', data)

def check_and_release_port(port):
    """檢查端口是否被占用，如果被占用則嘗試釋放"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.bind(("localhost", port))
            return True
        except socket.error:
            # 如果端口被占用，嘗試釋放它
            print(f"Port {port} is in use, attempting to release it...")
            if os.name == 'nt':
                # 在 Windows 上使用 netstat 和 taskkill
                result = subprocess.run(["netstat", "-ano"], capture_output=True, text=True, encoding='utf-8', errors='ignore')
                if result.stdout:
                    for line in result.stdout.splitlines():
                        if f":{port} " in line:
                            parts = line.split()
                            pid = parts[-1]
                            subprocess.run(["taskkill", "/PID", pid, "/F"])
                            print(f"Terminated process {pid} occupying port {port}")
            else:
                # 在 Unix 上使用 lsof 和 kill
                result = subprocess.run(["lsof", "-i", f":{port}"], capture_output=True, text=True, encoding='utf-8', errors='ignore')
                if result.stdout:
                    for line in result.stdout.splitlines():
                        if "LISTEN" in line:
                            parts = line.split()
                            pid = parts[1]
                            subprocess.run(["kill", "-9", pid])
                            print(f"Killed process {pid} occupying port {port}")
            return False

async def main(log_file):
    # 清空文件内容
    with open(log_file, "w", encoding='utf-8', errors='ignore') as f:
        f.write("")

    # 檢查並釋放端口
    port = 52045
    while not check_and_release_port(port):
        print(f"Waiting for port {port} to be released...")
        await asyncio.sleep(1)

    # 重定向 stdout 和 stderr
    sys.stdout = open(log_file, "a", encoding='utf-8', errors='ignore')
    sys.stderr = open(log_file, "a", encoding='utf-8', errors='ignore')

    config = uvicorn.Config(socket_app, host="localhost", port=port)
    server = uvicorn.Server(config)

    await server.serve()

def start_main_thread(log_file):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main(log_file))

# 移除多餘的 log_writer 並直接使用主執行緒
main_thread = threading.Thread(target=start_main_thread, args=("log.txt",))
main_thread.start()


In [2]:
class SocketTopic:
    user_message = 'user_message'
    doctor_message = "doctor_message" # 拼錯，應該是 doctor
    camera_toggle = "camera_toggle"
    report_toggle = "report_toggle"
    asr_toggle = "asr_toggle"

camera_toggle = False
report_toggle = False

In [34]:
camera_toggle = not camera_toggle
emit_message(SocketTopic.camera_toggle, camera_toggle)

In [24]:
report_toggle = not report_toggle
emit_message(SocketTopic.report_toggle, report_toggle)

In [10]:
text = "氣虛質的族群，常會有身體疲乏、喘促，以及多汗的情況，飲食上適合吃益氣健脾的食物，例如山藥、黃耆、小米、馬鈴薯、南瓜等，減少耗氣的食物如白蘿蔔、柚子等，有助改善這類情況"
audio_path = "C:\\Users\\User\\Documents\\work\\tcm-agent\\audio\\2024-06-22-16-49-15.wav"
message = {"audioPath" : audio_path, "text" : text}
emit_message(SocketTopic.doctor_message, message)